In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook demonstrates how to use EOTDL library to download Sentinel 2 images and automatically label them with AIS data.

Example developed from Agenium use case: https://github.com/earthpulse/eotdl/tree/main/tutorials/usecases/useCaseD